# Cleaning Earthquakes Raw Data

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine

POSTGRES_USER = os.environ.get("POSTGRES_USER")
POSTGRES_PASSWORD = os.environ.get("POSTGRES_PASSWORD")
POSTGRES_DB = os.environ.get("POSTGRES_DB")
POSTGRES_HOST = os.environ.get("POSTGRES_HOST")
POSTGRES_PORT = os.environ.get("POSTGRES_PORT")

conn = f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
engine = create_engine(conn)

In [22]:
query = "select * from raw_data;"

df = pd.read_sql_query(query, conn)


In [23]:
df.drop(columns=["code","event_id","url", "details"], axis=1, inplace=True)


In [24]:
df.set_index("id", inplace=True)

In [25]:
df

,place,city,country,magnitude,latitude,longitude,depth,utc_time
id,,,,,,,,
1,"14 km NNE of Virginia City, Nevada","Virginia City, Nevada",USA,1.0,39.421900,-119.557600,10.80,2024-01-27 11:27:58
2,"45 km NW of Toyah, Texas","Toyah, Texas",USA,1.9,31.619000,-104.117000,6.08,2024-01-27 11:39:20
3,"54 km NNE of Kobuk, Alaska","Kobuk, Alaska",USA,1.5,67.322500,-156.238600,4.40,2024-01-27 11:44:12
4,"279 km WNW of Houma, Tonga",Houma,Tonga,4.2,-20.158800,-177.761800,486.33,2024-01-27 11:50:11
5,"4 km SSW of Salcha, Alaska","Salcha, Alaska",USA,2.4,64.482800,-146.941300,8.50,2024-01-27 11:50:33
...,...,...,...,...,...,...,...,...
182031,"225 km W of Adak, Alaska","Adak, Alaska",USA,2.5,51.806667,-179.910333,15.64,2024-05-11 23:49:38
182032,"78 km W of Adak, Alaska","Adak, Alaska",USA,1.6,51.850500,-177.776167,3.17,2024-05-11 23:00:31
182033,"87 km NW of Karluk, Alaska","Karluk, Alaska",USA,1.0,58.177000,-155.392333,3.99,2024-05-11 19:05:53


## Remove region text from country

In [26]:
df["country"].str.contains("region")

df["country"] = df["country"].str.replace("region", "").str.strip()

df["country"].str.contains("region").sum()


0

In [27]:
df["country"].str.contains("of").sum()

pattern = r'north|south|east|west'

mask = df["country"].str.contains(pattern)
mask &= df["country"].str.contains("Islands")

df.loc[mask, "country"] = df.loc[mask, "country"].str.split().str[-2:].str.join(" ")


In [28]:
mask = df["place"].str.contains("Fiji") & df["country"].isnull()
df.loc[mask, "country"] = "Fiji"

In [29]:
mask = df["place"].str.contains("Fiji") & df["country"].str.contains("Islands")

df.loc[mask, "country"] = "Fiji"

mask.sum()

610

In [30]:
mask = df["country"].str.contains("Tonga") & df["country"].str.contains("of")

df.loc[mask, "country"] = "Tonga"

mask.sum()

51

In [31]:
mask = df["country"].str.contains("Alaska") | df["country"].str.contains("Texas") | df["country"].str.contains("California")

df.loc[mask, "country"] = "USA"

mask.sum()

3894

In [32]:
mask = df["city"].str.contains("Macquarie") & df["country"].str.contains("of")

df.loc[mask, "country"] = "Macquarie Island"

mask.sum()

51

In [33]:
mask = df["city"].str.contains("Ascension") & df["country"].str.contains("of")

df.loc[mask, "country"] = "Ascension Island"

mask.sum()

35

In [34]:
mask = df["city"].str.contains("America") & df["country"].str.contains("of")

df.loc[mask, "country"] = "Central America"

mask.sum()

40

In [35]:
to_change = ["Taiwan", "Venezuela", "Honduras", "Severnaya Zemlya", "Greenland", "Azerbaijan", "Guatemala", "Panama", "Svalbard", "Ecuador", "New Zealand", "Australia", "Chile", "Turkey", "Colombia", "Myanmar", "Oman", "Peru", "Japan", "Africa", "Nicaragua", "Franz Josef Land", "Syria", "Libya", "Easter Island"]

for country in to_change:
    mask = df["country"].str.contains(country) & df["country"].str.contains("of")

    df.loc[mask, "country"] = country
    print(f"{country} - {mask.sum()}")

Taiwan - 3
Venezuela - 3
Honduras - 1
Severnaya Zemlya - 19
Greenland - 3
Azerbaijan - 1
Guatemala - 7
Panama - 18
Svalbard - 18
Ecuador - 13
New Zealand - 28
Australia - 6
Chile - 1
Turkey - 1
Colombia - 2
Myanmar - 1
Oman - 1
Peru - 12
Japan - 2
Africa - 34
Nicaragua - 8
Franz Josef Land - 2
Syria - 1
Libya - 2
Easter Island - 38


In [36]:
df["country"].isnull().sum()



0

In [37]:
df["country"].value_counts().head(25)

country
USA                         146560
Puerto Rico                   5972
Indonesia                     2787
Japan                         2057
Philippines                   1794
Fiji                          1473
Papua New Guinea              1193
Mexico                        1100
Tonga                         1078
Chile                          984
Canada                         813
Vanuatu                        730
Russia                         702
Northern Mariana Islands       643
Turkey                         638
South Sandwich Islands         612
Kermadec Islands               572
China                          517
U.S. Virgin Islands            501
Reykjanes Ridge                476
Taiwan                         460
Argentina                      454
New Zealand                    416
Peru                           413
Solomon Islands                409
Name: count, dtype: int64

In [38]:
new_table = "updated_data"

df.to_sql(new_table, engine, if_exists="replace")

35

In [39]:
engine.dispose()